In [113]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import gensim
from gensim.models import Word2Vec, KeyedVectors
from gensim.utils import simple_preprocess
import spacy
import en_core_web_sm
from nltk import sent_tokenize
from tqdm import tqdm
import pickle


In [114]:
temp_df = pd.read_csv('Datasets\\IMDB Dataset.csv')

# Data Preprocessing

In [115]:
df = temp_df.iloc[:10000]

In [116]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [117]:
df['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [118]:
df['sentiment'].value_counts()

sentiment
positive    5028
negative    4972
Name: count, dtype: int64

In [119]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [120]:
df.duplicated().sum()

17

In [121]:
df.drop_duplicates(inplace=True)

C:\Users\Rohan\AppData\Local\Temp\ipykernel_7912\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [122]:
df.duplicated().sum()

0

In [123]:
# Basic Preprocessing
# Remove tags
# lowercase
# remove stopwords

In [124]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [125]:
df['review'] = df['review'].apply(remove_tags)

C:\Users\Rohan\AppData\Local\Temp\ipykernel_7912\2336150696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [126]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
9995,"Fun, entertaining movie about WWII German spy ...",positive
9996,Give me a break. How can anyone say that this ...,negative
9997,This movie is a bad movie. But after watching ...,negative
9998,This is a movie that was probably made to ente...,negative


In [127]:
df['review'] = df['review'].apply(lambda x:x.lower())

C:\Users\Rohan\AppData\Local\Temp\ipykernel_7912\740760900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [128]:
sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

C:\Users\Rohan\AppData\Local\Temp\ipykernel_7912\1617134234.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [129]:
df

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production. filming technique...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there's family little boy (jake) thi...,negative
4,"petter mattei's ""love time money"" visually stu...",positive
...,...,...
9995,"fun, entertaining movie wwii german spy (julie...",positive
9996,"give break. anyone say ""good hockey movie""? kn...",negative
9997,movie bad movie. watching endless series bad h...,negative
9998,"movie probably made entertain middle school, e...",negative


## Splitting Data into Target and Features

In [130]:
X = df.iloc[:,0:1]
y = df['sentiment']

In [131]:
X

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."
...,...
9995,"fun, entertaining movie wwii german spy (julie..."
9996,"give break. anyone say ""good hockey movie""? kn..."
9997,movie bad movie. watching endless series bad h...
9998,"movie probably made entertain middle school, e..."


In [132]:
y

0       positive
1       positive
2       positive
3       negative
4       positive
          ...   
9995    positive
9996    negative
9997    negative
9998    negative
9999    positive
Name: sentiment, Length: 9983, dtype: object

## Label Encoding

In [133]:
encoder = LabelEncoder()

y = encoder.fit_transform(y)

In [134]:
y

array([1, 1, 1, ..., 0, 0, 1])

1 --> positive

0 --> negative

## Spliting data into training and testing data

In [135]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [136]:
X_train.shape

(7986, 1)

# Word Embedding

## Using Bow

In [137]:
cv = CountVectorizer()

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

In [138]:
# Gaussian Naive Bayes
gnb = GaussianNB()

gnb.fit(X_train_bow,y_train)
y_pred = gnb.predict(X_test_bow)
round(accuracy_score(y_test,y_pred)*100,2)

63.24

In [139]:
confusion_matrix(y_test,y_pred)

array([[717, 235],
       [499, 546]], dtype=int64)

In [140]:
# Random Forest Classifier
rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
round(accuracy_score(y_test,y_pred)*100,2)

84.63

In [141]:
confusion_matrix(y_test,y_pred)

array([[801, 151],
       [156, 889]], dtype=int64)

## Using Bow (with first 3000 max frequent features)

In [142]:
cv = CountVectorizer(max_features=3000)

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
round(accuracy_score(y_test,y_pred)*100,2)

83.98

## Using Bow (with first 5000 max frequent features)

In [143]:
cv = CountVectorizer(ngram_range=(1,2),max_features=5000)

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
round(accuracy_score(y_test,y_pred)*100,2)

84.53

In [144]:
# Save the trained model
with open('movie_review_sentiment_analysis_model.pkl', 'wb') as file:
    pickle.dump(rf, file)

# Save the fitted CountVectorizer
with open('count_vectorizer.pkl', 'wb') as file:
    pickle.dump(cv, file)

## Using TF-TDF

In [145]:
tfidf = TfidfVectorizer()

X_train_tfidf = tfidf.fit_transform(X_train['review']).toarray()
X_test_tfidf = tfidf.transform(X_test['review'])

rf = RandomForestClassifier()

rf.fit(X_train_tfidf,y_train)
y_pred = rf.predict(X_test_tfidf)
round(accuracy_score(y_test,y_pred)*100,2)

84.48

## Using Google News Vectors

In [146]:
model = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz",binary=True,limit=500000)

In [147]:
model['cricket'].shape

(300,)

In [148]:
sw_list = stopwords.words('english')
sw_list

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [149]:
# Remove stopwords
X_train = X_train['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))
X_test = X_test['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [150]:
nlp = en_core_web_sm.load()
doc = nlp(X_train.values[0])
print(doc.vector)

[-0.12258767 -0.32986197 -0.08605512 -0.08838645  0.02692593  0.34582236
  0.24921143  0.24986878  0.2088419  -0.20151913  0.23178497 -0.1762677
 -0.1071358  -0.06781842 -0.10437153  0.21653889 -0.06587398 -0.12128343
  0.25721326 -0.24933322 -0.24735817  0.42293647 -0.01461194  0.0914708
  0.07562461  0.08937843  0.143013    0.25307712  0.15811773  0.17392442
  0.15753338 -0.01662673 -0.06337961  0.01045302 -0.22417541 -0.1052129
 -0.0704873   0.07822629 -0.20214622 -0.17670628 -0.11396704  0.20277347
 -0.1665241   0.06917517  0.11663126  0.20812093 -0.05610794  0.03596068
  0.13438044 -0.04062429 -0.1362053   0.0959331   0.00262649 -0.06334188
 -0.05186576 -0.0642135   0.03902575  0.00546995 -0.05921976  0.07042424
  0.03771723  0.01412539 -0.03606035 -0.32633492 -0.03871979  0.05871588
  0.15013589 -0.08593327 -0.04132756 -0.15561156  0.19222257  0.12445915
 -0.02280001 -0.21367836  0.17164145 -0.00392572 -0.18648992 -0.20370802
  0.07489342 -0.35127598 -0.10086565 -0.02948266 -0.22

In [151]:
input_arr = []
for item in X_train.values:
    doc = nlp(item)
    input_arr.append(doc.vector)

In [152]:
input_arr = np.array(input_arr)

In [153]:
input_arr.shape

(7986, 96)

In [154]:
input_test_arr = []
for item in X_test.values:
    doc = nlp(item)
    input_test_arr.append(doc.vector)

In [155]:
input_test_arr = np.array(input_test_arr)

In [156]:
input_test_arr.shape

(1997, 96)

In [157]:
gnb = GaussianNB()

gnb.fit(input_arr,y_train)
y_pred = gnb.predict(input_test_arr)
round(accuracy_score(y_test,y_pred)*100,2)

60.89

## Using Word2Vec

In [158]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))
    

In [159]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [160]:
model.build_vocab(story)

In [161]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5875663, 6212140)

In [162]:
len(model.wv.index_to_key)

31845

In [163]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [164]:
document_vector(df['review'].values[0])

array([-0.17771924,  0.46856016,  0.13524057,  0.23843555, -0.13761498,
       -0.6271021 ,  0.21203911,  0.92201066, -0.34909022, -0.28503758,
       -0.29183054, -0.49582142,  0.11604077,  0.07682965,  0.16367297,
       -0.089766  ,  0.0097013 , -0.34710732, -0.06836189, -0.6470112 ,
        0.06168715,  0.22329995,  0.11111229, -0.2763144 , -0.338013  ,
       -0.01407239, -0.34052387,  0.00258142, -0.35239595,  0.0122604 ,
        0.35508123,  0.03275272,  0.23751225, -0.2872752 , -0.10950251,
        0.41568932,  0.10134548, -0.3888308 , -0.24487929, -0.7483098 ,
        0.09926604, -0.25826022,  0.09353512, -0.06957095,  0.48444194,
       -0.20511217, -0.24937311, -0.01925628,  0.11785182,  0.37629047,
        0.11613186, -0.3543568 , -0.38605046, -0.10536385, -0.1189503 ,
        0.19745983,  0.245799  ,  0.01242109, -0.28684577,  0.12603164,
        0.10086833,  0.10136463,  0.04058383, -0.05183543, -0.45685792,
        0.26433176,  0.04229804,  0.09579326, -0.30601007,  0.28

In [165]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████| 9983/9983 [10:37<00:00, 15.67it/s]


In [166]:
X = np.array(X)
X[0]

array([-0.17771924,  0.46856016,  0.13524057,  0.23843555, -0.13761498,
       -0.6271021 ,  0.21203911,  0.92201066, -0.34909022, -0.28503758,
       -0.29183054, -0.49582142,  0.11604077,  0.07682965,  0.16367297,
       -0.089766  ,  0.0097013 , -0.34710732, -0.06836189, -0.6470112 ,
        0.06168715,  0.22329995,  0.11111229, -0.2763144 , -0.338013  ,
       -0.01407239, -0.34052387,  0.00258142, -0.35239595,  0.0122604 ,
        0.35508123,  0.03275272,  0.23751225, -0.2872752 , -0.10950251,
        0.41568932,  0.10134548, -0.3888308 , -0.24487929, -0.7483098 ,
        0.09926604, -0.25826022,  0.09353512, -0.06957095,  0.48444194,
       -0.20511217, -0.24937311, -0.01925628,  0.11785182,  0.37629047,
        0.11613186, -0.3543568 , -0.38605046, -0.10536385, -0.1189503 ,
        0.19745983,  0.245799  ,  0.01242109, -0.28684577,  0.12603164,
        0.10086833,  0.10136463,  0.04058383, -0.05183543, -0.45685792,
        0.26433176,  0.04229804,  0.09579326, -0.30601007,  0.28

In [167]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)


In [168]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
round(accuracy_score(y_test,y_pred)*100,2)

76.72